In [3]:
!pip install -U -q  Xodia24 stable_baselines3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.1/182.1 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 9.3 MB/s eta 0:00:00


In [4]:
from Xodia24.env import PocketTank

In [5]:
pt = PocketTank()


In [6]:
from stable_baselines3 import PPO
import os
# from train_env import TrainEnv
from stable_baselines3.common.env_checker import check_env
import torch as th


# Creating a subclass of Pocket Tanks for customizing the reward fucntion
class CustomEnv(PocketTank):
    def __init__(self):
        super().__init__()
        self.action_cnt=100

    def _get_reward(self, diff, bullet_type):
        x = diff
      #Write your own reward function, this is a dummy function only

        # Standard Shell -> 20 damage
        if(bullet_type==0):

            # y=-0.04x^{2}+160
            if(x<=25):
                return (-0.04*(x*x)+160)
                # max_reward = 160
            else:
                return -60

        # Triple Threat -> each bullet 20 damage
        elif(bullet_type==1):
            # y=-0.0005x^{2}+90
            if(x<=240):
                return (-0.0005*(x*x))+90
            else:
                return -50

        # Long Shot -> based on distance travelled
        elif(bullet_type==2):
            # y=0.25x
            # depends whether the range of both the tanks is fixed or not (considering the fixed limits)
            # if(500<=x<=750):
            #     return (0.24*x)
            # else:
            #     return -30
            if(x<=200):
                return (-0.5*x)+100
            else:
                return -20


        # Heavy Impact -> limited range and velocity
        elif(bullet_type==3):
            # damage : 40
            if(x<=250):
                # y=-0.002x^{2}+200 ->   max=200
                return (-0.002*(x*x))+200
            else:
                return -100

        # Blast Radius -> within a radius of 100
        elif(bullet_type==4):
            # damage : 10
            if(x<=80):
                # y=-0.003x^{2}+100
                return (-0.003*(x*x))+100
            elif(80<x<=100):
                return 60
            elif(100<x<110):
                return -20
            else:
                return -30

        # Healing Halo -> heals by 10 pts
        elif(bullet_type==5):
            # y=-0.17x^{2}+130
            if(x<=25):
                # y=-0.04x^{2}+130
                return (-0.04*(x*x)+130)
                # max_reward = 130
            elif(diff<=28):
                return 40
            else:
                return -80

        # Boomerang Blast -> 20 damage
        elif(bullet_type==6):
            if(x<=200):
                # y=\frac{1200}{x+10}+100 -> max 300 min
                return (1200/(x+10))+100
            else:
                return -70

        else:
            print("Invalid bullet type")



In [ ]:
models_dir = "models/PPO"
logs_dir = "logs"

if not os.path.exists(models_dir):
    os.makedirs(models_dir)

if not os.path.exists(logs_dir):
    os.makedirs(logs_dir)

env = CustomEnv()
policy_kwargs = dict(activation_fn=th.nn.Tanh,
                     net_arch=dict(pi=[92, 92], vf=[92, 92]))

bot = PPO(policy='MlpPolicy', env = env, verbose= 1, ent_coef=0.5,tensorboard_log=logs_dir)

check_env(env)


TIMESTEPS = 25000
for i in range(1,20):
    bot.learn(total_timesteps=TIMESTEPS,progress_bar=True,reset_num_timesteps=False,tb_log_name = "PPO")  # training the model
    bot.save(f"{models_dir}/{TIMESTEPS*i}")



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to logs/PPO_0


Output()

----------------------------------
| rollout/           |           |
|    ep_len_mean     | 100       |
|    ep_rew_mean     | -3.59e+03 |
| time/              |           |
|    fps             | 132       |
|    iterations      | 1         |
|    time_elapsed    | 15        |
|    total_timesteps | 2048      |
----------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 100         |
|    ep_rew_mean          | -3.67e+03   |
| time/                   |             |
|    fps                  | 146         |
|    iterations           | 2           |
|    time_elapsed         | 27          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.000103717 |
|    clip_fraction        | 0           |
|    clip_range           | 0.2         |
|    entropy_loss         | -10.2       |
|    explained_variance   | 5.84e-06    |
|    learning_rate  

Logging to logs/PPO_0


...

Output()

----------------------------------
| rollout/           |           |
|    ep_len_mean     | 100       |
|    ep_rew_mean     | -2.68e+03 |
| time/              |           |
|    fps             | 237       |
|    iterations      | 1         |
|    time_elapsed    | 8         |
|    total_timesteps | 215040    |
----------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 100          |
|    ep_rew_mean          | -2.69e+03    |
| time/                   |              |
|    fps                  | 204          |
|    iterations           | 2            |
|    time_elapsed         | 20           |
|    total_timesteps      | 217088       |
| train/                  |              |
|    approx_kl            | 0.0015292247 |
|    clip_fraction        | 0.00386      |
|    clip_range           | 0.2          |
|    entropy_loss         | -10.2        |
|    explained_variance   | -1.19e-07    |
|    

Logging to logs/PPO_0


Output()

----------------------------------
| rollout/           |           |
|    ep_len_mean     | 100       |
|    ep_rew_mean     | -2.68e+03 |
| time/              |           |
|    fps             | 238       |
|    iterations      | 1         |
|    time_elapsed    | 8         |
|    total_timesteps | 241664    |
----------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 100          |
|    ep_rew_mean          | -2.76e+03    |
| time/                   |              |
|    fps                  | 203          |
|    iterations           | 2            |
|    time_elapsed         | 20           |
|    total_timesteps      | 243712       |
| train/                  |              |
|    approx_kl            | 0.0014359439 |
|    clip_fraction        | 0.000244     |
|    clip_range           | 0.2          |
|    entropy_loss         | -10.2        |
|    explained_variance   | 0            |
|    